In [2]:
import json
import os
import sys
from copy import deepcopy
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import skimage
import seaborn as sns
import tifffile
import yaml

In [3]:
%load_ext autoreload

In [4]:
%autoreload 2

In [5]:
from mushroom.mushroom import Mushroom, DEFAULT_CONFIG
import mushroom.data.he as he
import mushroom.data.multiplex as multiplex
import mushroom.data.visium as visium
import mushroom.utils as utils
import mushroom.visualization.utils as vis_utils

In [6]:
source_root = '/diskmnt/Projects/Users/estorrs/mushroom/data'
target_root = '/data/estorrs/mushroom/data'

In [7]:
run_dir = '/data/estorrs/mushroom/data/projects/submission_v1/kidney/'

## define inputs

In [14]:
fps = sorted(utils.listfiles(run_dir, regex=r'registered/metadata.yaml$'))
fps

['/data/estorrs/mushroom/data/projects/submission_v1/kidney/NMK12F-Fp1/registered/metadata.yaml',
 '/data/estorrs/mushroom/data/projects/submission_v1/kidney/NMK12M-Fp1/registered/metadata.yaml',
 '/data/estorrs/mushroom/data/projects/submission_v1/kidney/NMK3F-Fp1/registered/metadata.yaml',
 '/data/estorrs/mushroom/data/projects/submission_v1/kidney/NMK3M-Fp1/registered/metadata.yaml',
 '/data/estorrs/mushroom/data/projects/submission_v1/kidney/NMK92F-Fp1/registered/metadata.yaml',
 '/data/estorrs/mushroom/data/projects/submission_v1/kidney/NMK92F2-Fc1U1Bs2/registered/metadata.yaml',
 '/data/estorrs/mushroom/data/projects/submission_v1/kidney/NMK92M-Fp1/registered/metadata.yaml',
 '/data/estorrs/mushroom/data/projects/submission_v1/kidney/NMK92M1-Fc1U1Bs2/registered/metadata.yaml',
 '/data/estorrs/mushroom/data/projects/submission_v1/kidney/P1F1MnR-Fp1/registered/metadata.yaml',
 '/data/estorrs/mushroom/data/projects/submission_v1/kidney/P1M3MnR-Fp1/registered/metadata.yaml',
 '/data/

In [15]:
# fps = [
#     '/data/estorrs/mushroom/data/projects/submission_v1/kidney/P21_F2RM6R/registered/metadata.yaml'
# ]
# fps

In [16]:
DEFAULT_CONFIG

{'sections': None,
 'dtype_to_chkpt': None,
 'dtype_specific_params': {'visium': {'trainer_kwargs': {'tiling_method': 'radius'}}},
 'sae_kwargs': {'size': 8,
  'patch_size': 1,
  'encoder_dim': 128,
  'codebook_dim': 64,
  'num_clusters': (8, 4, 2),
  'dtype_to_decoder_dims': {'multiplex': (256, 128, 64),
   'he': (256, 128, 10),
   'visium': (256, 512, 2048),
   'xenium': (256, 256, 256),
   'cosmx': (256, 512, 1024),
   'points': (256, 512, 1024)},
  'recon_scaler': 1.0,
  'neigh_scaler': 0.01},
 'trainer_kwargs': {'input_resolution': 1.0,
  'target_resolution': 0.02,
  'pct_expression': 0.05,
  'log_base': 2.718281828459045,
  'tiling_method': 'grid',
  'tiling_radius': 1.0,
  'batch_size': 128,
  'num_workers': 0,
  'devices': 1,
  'accelerator': 'cpu',
  'max_epochs': 1,
  'steps_per_epoch': 1000,
  'lr': 0.0001,
  'out_dir': './outputs',
  'save_every': 1,
  'log_every_n_steps': 10,
  'logger_type': 'tensorboard',
  'logger_project': 'portobello',
  'channel_mapping': {},
  'data

In [17]:
def alter_filesystem(config, source_root, target_root):
    for entry in config['sections']:
        for mapping in entry['data']:
            mapping['filepath'] = mapping['filepath'].replace(source_root, target_root)
    
    if config['trainer_kwargs']['data_mask'] is not None:
        config['trainer_kwargs']['data_mask'] = config['trainer_kwargs']['data_mask'].replace(source_root, target_root)
        
    return config

In [18]:
case_to_config = {}

for fp in fps:
    metadata = yaml.safe_load(open(fp))
    
    case = fp.split('/')[-3]
    
    config = deepcopy(DEFAULT_CONFIG)
    config = utils.recursive_update(config, {
        'sections': metadata['sections'],
        'trainer_kwargs': {
            'input_resolution': metadata['resolution'],
            'target_resolution': 50, # 50 ppm
            'out_dir': os.path.join(run_dir, case, 'mushroom'),
            'accelerator': 'gpu',
            'steps_per_epoch': 1000,
        }
    })
    
    config['dtype_specific_params'] = utils.recursive_update(config['dtype_specific_params'], {
        'visium': {
            'trainer_kwargs': {
                'target_resolution': 100  # do visium at lower res
            }
        }
    })
    
    config = alter_filesystem(config, source_root, target_root)
    
    case_to_config[case] = config

In [19]:
case_to_config.keys()

dict_keys(['NMK12F-Fp1', 'NMK12M-Fp1', 'NMK3F-Fp1', 'NMK3M-Fp1', 'NMK92F-Fp1', 'NMK92F2-Fc1U1Bs2', 'NMK92M-Fp1', 'NMK92M1-Fc1U1Bs2', 'P1F1MnR-Fp1', 'P1M3MnR-Fp1', 'P1_F1LM3l', 'P21_F2RM6R'])

In [20]:
case_to_config

{'NMK12F-Fp1': {'sections': [{'data': [{'dtype': 'xenium',
      'filepath': '/data/estorrs/mushroom/data/projects/submission_v1/kidney/NMK12F-Fp1/registered/NMK12F-Fp1_xenium.h5ad'}],
    'position': 0,
    'sid': 'NMK12F-Fp1-U1'}],
  'dtype_to_chkpt': None,
  'dtype_specific_params': {'visium': {'trainer_kwargs': {'tiling_method': 'radius',
     'target_resolution': 100}}},
  'sae_kwargs': {'size': 8,
   'patch_size': 1,
   'encoder_dim': 128,
   'codebook_dim': 64,
   'num_clusters': (8, 4, 2),
   'dtype_to_decoder_dims': {'multiplex': (256, 128, 64),
    'he': (256, 128, 10),
    'visium': (256, 512, 2048),
    'xenium': (256, 256, 256),
    'cosmx': (256, 512, 1024),
    'points': (256, 512, 1024)},
   'recon_scaler': 1.0,
   'neigh_scaler': 0.01},
  'trainer_kwargs': {'input_resolution': 1.0,
   'target_resolution': 50,
   'pct_expression': 0.05,
   'log_base': 2.718281828459045,
   'tiling_method': 'grid',
   'tiling_radius': 1.0,
   'batch_size': 128,
   'num_workers': 0,
   'd

In [13]:
completed = []

# completed = [
#     'NMK12F-Fp1',
#     'NMK12M-Fp1',
#     'NMK3F-Fp1',
#     'NMK3M-Fp1',
#     'NMK92F-Fp1',
# ]

In [1]:
# for case, config in case_to_config.items():
#     if case not in completed:
#         print(case)
#         mushroom = Mushroom.from_config(config)
#         mushroom.train()
#         mushroom.embed_sections()
#         mushroom.save()

#         del(mushroom)

###### multi-res for best cases

In [21]:
target_resolutions = [25, 50, 100, 200]
keep = [
    'P21_F2RM6R',
]

In [22]:
for case in keep:
    config = deepcopy(case_to_config[case])
    for res in target_resolutions:
        config['trainer_kwargs']['target_resolution'] = res
        config['trainer_kwargs']['out_dir'] = os.path.join(run_dir, case, f'mushroom_{res}res')
        print(case, res)
        mushroom = Mushroom.from_config(config)
        mushroom.train()
        mushroom.embed_sections()
        mushroom.save()

        del(mushroom)

INFO:root:loading spore for multiplex
INFO:root:singleton section detected, creating temporary duplicate
INFO:root:data mask detected
INFO:root:starting multiplex processing
INFO:root:using 27 channels
INFO:root:2 sections detected: ['P21_F2RM6R-U1', 'P21_F2RM6R-U1_dup']
INFO:root:processing sections
INFO:root:generating image data for section P21_F2RM6R-U1


P21_F2RM6R 25
0.020403050108932463


INFO:root:generating image data for section P21_F2RM6R-U1_dup
INFO:root:generating training dataset
INFO:root:generating inference dataset
INFO:root:total of 2 sections detected: [('P21_F2RM6R-U1', 'multiplex'), ('P21_F2RM6R-U1_dup', 'multiplex')]
INFO:root:creating data loaders
INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmpjzeodiqw
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmpjzeodiqw/_remote_module_non_scriptable.py
INFO:root:creating ViT
INFO:root:model initialized


multiplex torch.Size([27, 285, 183])


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
INFO:root:starting training for multiplex
Missing logger folder: /data/estorrs/mushroom/data/projects/submission_v1/kidney/P21_F2RM6R/mushroom_25res/multiplex_logs/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name | Type | Params
------------------------------
0 | sae  | SAE  | 5.0 M 
------------------------------
5.0 M     Trainable params
0         Non-trainable params
5.0 M     Total params
19.915    Total estimated model params size (MB)
/data/estorrs/miniconda3/envs/mushroom/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.


...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name | Type | Params
------------------------------
0 | sae  | SAE  | 5.0 M 
------------------------------
5.0 M     Trainable params
0         Non-trainable params
5.0 M     Total params
19.915    Total estimated model params size (MB)
/data/estorrs/miniconda3/envs/mushroom/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.


Training: |                                                                                                   …

`Trainer.fit` stopped: `max_epochs=1` reached.
INFO:root:finished training multiplex, saved chkpt to /data/estorrs/mushroom/data/projects/submission_v1/kidney/P21_F2RM6R/mushroom_200res/multiplex_chkpts/last.ckpt
INFO:root:embedding multiplex spore
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
/data/estorrs/miniconda3/envs/mushroom/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.


Predicting: |                                                                                                 …

INFO:root:saving config and outputs to /data/estorrs/mushroom/data/projects/submission_v1/kidney/P21_F2RM6R/mushroom_200res


In [23]:
target_resolutions = [.001, .002, .004, .006]
keep = [
    'NMK92F2-Fc1U1Bs2'
]

In [ ]:
for case in keep:
    config = deepcopy(case_to_config[case])
    for res in target_resolutions:
        config['trainer_kwargs']['target_resolution'] = res
        config['trainer_kwargs']['out_dir'] = os.path.join(run_dir, case, f'mushroom_{res}res')
        config['dtype_specific_params'] = utils.recursive_update(config['dtype_specific_params'], {
            'visium': {
                'trainer_kwargs': {
                    'target_resolution': res
                }
            }
        })

        print(case, res)
        mushroom = Mushroom.from_config(config)
        mushroom.train()
        mushroom.embed_sections()
        mushroom.save()

        del(mushroom)

INFO:root:loading spore for visium
INFO:root:singleton section detected, creating temporary duplicate
INFO:root:data mask detected
INFO:root:starting visium processing


NMK92F2-Fc1U1Bs2 0.001


INFO:root:using 11508 channels
INFO:root:2 sections detected: ['NMK92F2-Fc1U1Bs2-U1', 'NMK92F2-Fc1U1Bs2-U1_dup']
INFO:root:processing sections
INFO:root:generating image data for section NMK92F2-Fc1U1Bs2-U1
INFO:root:generating image data for section NMK92F2-Fc1U1Bs2-U1_dup
INFO:root:generating training dataset
INFO:root:generating inference dataset
INFO:root:total of 2 sections detected: [('NMK92F2-Fc1U1Bs2-U1', 'visium'), ('NMK92F2-Fc1U1Bs2-U1_dup', 'visium')]
INFO:root:creating data loaders
INFO:root:creating ViT
INFO:root:model initialized
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
INFO:root:starting training for visium
Missing logger folder: /data/estorrs/mushroom/data/projects/submission_v1/kidney/NMK92F2-Fc1U1Bs2/mushroom_0.001res/visium_logs/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name | Type | Params
------------------------------
0 | sae  | 

Training: |                                                                                                   …

`Trainer.fit` stopped: `max_epochs=1` reached.
INFO:root:finished training visium, saved chkpt to /data/estorrs/mushroom/data/projects/submission_v1/kidney/NMK92F2-Fc1U1Bs2/mushroom_0.001res/visium_chkpts/last.ckpt
INFO:root:embedding visium spore
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
/data/estorrs/miniconda3/envs/mushroom/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.


...

NMK92F2-Fc1U1Bs2 0.006


INFO:root:using 11508 channels
INFO:root:2 sections detected: ['NMK92F2-Fc1U1Bs2-U1', 'NMK92F2-Fc1U1Bs2-U1_dup']
INFO:root:processing sections
INFO:root:generating image data for section NMK92F2-Fc1U1Bs2-U1
INFO:root:generating image data for section NMK92F2-Fc1U1Bs2-U1_dup
INFO:root:generating training dataset
INFO:root:generating inference dataset
INFO:root:total of 2 sections detected: [('NMK92F2-Fc1U1Bs2-U1', 'visium'), ('NMK92F2-Fc1U1Bs2-U1_dup', 'visium')]
INFO:root:creating data loaders
INFO:root:creating ViT
INFO:root:model initialized
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
INFO:root:starting training for visium
Missing logger folder: /data/estorrs/mushroom/data/projects/submission_v1/kidney/NMK92F2-Fc1U1Bs2/mushroom_0.006res/visium_logs/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name | Type | Params
------------------------------
0 | sae  | 

Training: |                                                                                                   …

`Trainer.fit` stopped: `max_epochs=1` reached.
INFO:root:finished training visium, saved chkpt to /data/estorrs/mushroom/data/projects/submission_v1/kidney/NMK92F2-Fc1U1Bs2/mushroom_0.006res/visium_chkpts/last.ckpt
INFO:root:embedding visium spore
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
/data/estorrs/miniconda3/envs/mushroom/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.


Predicting: |                                                                                                 …